## Задание - Классификация: Логистическая регрессия и SVM

Имеются данные adult.csv (см. в материалах к занятию).

Целевой переменной является уровень дохода income (крайний правый столбец).

Описание признаков:

- **age**: continuous.
- **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- **fnlwgt**: continuous.
- **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- **education-num**: continuous.
- **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-- inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- **race**: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- **sex**: Female, Male.
- **capital-gain**: continuous.
- **capital-loss**: continuous.
- **hours-per-week**: continuous.
- **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- **income**: >50K, <=50K

Задача:
1. Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.
2. Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

## Решение

In [1]:
import pandas
import numpy

In [2]:
def get_adult_data():
    data_frame = pandas.read_csv('adult.csv')
    
    for column in data_frame.columns:
        data_frame = data_frame.loc[data_frame[column] != '?']
        
    return data_frame

adult_data = get_adult_data()
adult_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              45222 non-null  int64 
 1   workclass        45222 non-null  object
 2   fnlwgt           45222 non-null  int64 
 3   education        45222 non-null  object
 4   educational-num  45222 non-null  int64 
 5   marital-status   45222 non-null  object
 6   occupation       45222 non-null  object
 7   relationship     45222 non-null  object
 8   race             45222 non-null  object
 9   gender           45222 non-null  object
 10  capital-gain     45222 non-null  int64 
 11  capital-loss     45222 non-null  int64 
 12  hours-per-week   45222 non-null  int64 
 13  native-country   45222 non-null  object
 14  income           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [3]:
adult_data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K


**Подготовка данных**

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def tramsform_categorial_columns_to_numeric(data_frame):
    '''
    Переводит категориальные колонки в числовые
    '''
    columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country', 'gender']
    tramsform_data = pandas.get_dummies(
        data_frame, 
        columns=columns)
    
    return tramsform_data
    
def split_data(data_frame):
    '''
    Разделяет данные на тренировачные и тестовые.
    
    Return: X_train, X_test, y_train, y_test
    '''
    adult_data_train, adult_data_test = train_test_split(
        numeric_adult_data, 
        test_size=0.2)
    income_key = 'income'
    y_train = get_to_binar_column(adult_data_train, income_key)
    y_test = get_to_binar_column(adult_data_test, income_key)
    
    del adult_data_train[income_key]
    del adult_data_test[income_key]
    
    return adult_data_train, adult_data_test, y_train, y_test

def get_to_binar_column(data_frame, column_name):
    '''
    Переводит категориальную колонку в бинарную
    '''
    gender_label_encoder = LabelEncoder()
    column = data_frame[column_name]
    gender_label_encoder.fit(column)
    return pandas.Series(data=gender_label_encoder.transform(column))

numeric_adult_data = tramsform_categorial_columns_to_numeric(data_frame=adult_data)
adult_X_train, adult_X_test, adult_y_train, adult_y_test = split_data(numeric_adult_data)

#### Логическая регрессия

In [5]:
from sklearn.linear_model import LogisticRegression

logisticRegression = LogisticRegression(solver='lbfgs', max_iter=10000)
logisticRegression.fit(adult_X_train, adult_y_train)

LogisticRegression(max_iter=10000)

In [6]:
logisticRegression.score(adult_X_train, adult_y_train)

0.7908892390192663

In [7]:
logisticRegression.score(adult_X_test, adult_y_test)

0.7928137092316196

#### SVM

In [8]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = make_pipeline(StandardScaler(), SVC(gamma='auto', max_iter=10000))
model.fit(adult_X_train, adult_y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/svm/_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto', max_iter=10000))])

In [9]:
model.score(adult_X_train, adult_y_train)

0.8580589877546507

In [10]:
model.score(adult_X_test, adult_y_test)

0.8506357103372029